## Encoding Numerical Features

### Binarization

In [2]:
import numpy as np
import pandas as pd

In [15]:
df = pd.read_csv('C:\\Data Science Preparation\\1_CampusX - end to end\\100 Days of Machine Learning\\Day19-22\\train.csv', usecols=['Age', 'Fare', 'SibSp', 'Parch', 'Survived'])
df.dropna(inplace=True) # we are dropping all null values
df.head()

,Survived,Age,SibSp,Parch,Fare
0,0,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,1,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,0,35.0,0,0,8.0500


In [16]:
# feature engineering
df['family'] = df['SibSp'] + df['Parch']
df.head()

,Survived,Age,SibSp,Parch,Fare,family
0,0,22.0,1,0,7.2500,1
1,1,38.0,1,0,71.2833,1
2,1,26.0,0,0,7.9250,0
3,1,35.0,1,0,53.1000,1
4,0,35.0,0,0,8.0500,0


In [17]:
# drop the old columns
df.drop(columns=['SibSp', 'Parch'], inplace=True)
df.head()

,Survived,Age,Fare,family
0,0,22.0,7.2500,1
1,1,38.0,71.2833,1
2,1,26.0,7.9250,0
3,1,35.0,53.1000,1
4,0,35.0,8.0500,0


In [19]:
x = df.drop(columns=['Survived'])
y = df['Survived']

In [22]:
# train test split
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train.head()

,Age,Fare,family
328,31.0,20.5250,2
73,26.0,14.4542,1
253,30.0,16.1000,1
719,33.0,7.7750,0
666,25.0,13.0000,0


### Without Binarization

In [24]:
# Apply Model without Binarization
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()
clf.fit(x_train, y_train)

y_pred = clf.predict(x_test)

from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.6433566433566433

In [26]:
# cross validation
from sklearn.model_selection import cross_val_score
np.mean(cross_val_score(DecisionTreeClassifier(), x, y, cv=10, scoring='accuracy'))

0.6457159624413146

### With Binarization

In [28]:
from sklearn.preprocessing import Binarizer
from sklearn.compose import ColumnTransformer

In [36]:
# create a transformer object
trf = ColumnTransformer([
    ('bin', Binarizer(copy=False), ['family'])
], remainder='passthrough')
# threshold default is 0, and we want to make changes to the existing columns

In [37]:
# transforme the datasets
x_train_trf = trf.fit_transform(x_train)
x_test_trf = trf.transform(x_test)

In [39]:
# since x_train_trf is a numpy array, we covert it DataFrame with these columns
pd.DataFrame(x_train_trf, columns=['famliy', 'Age', 'Fare'])

,famliy,Age,Fare
0,1.0,31.0,20.5250
1,1.0,26.0,14.4542
2,1.0,30.0,16.1000
3,0.0,33.0,7.7750
4,0.0,25.0,13.0000
...,...,...,...
566,1.0,46.0,61.1750
567,0.0,25.0,13.0000
568,0.0,41.0,134.5000
569,1.0,33.0,20.5250


In [35]:
# now apply the model
clf = DecisionTreeClassifier()
clf.fit(x_train_trf, y_train)
y_pred2 = clf.predict(x_test_trf)

accuracy_score(y_test, y_pred2)

0.6013986013986014

In [41]:
# cross validation
x_trf = trf.fit_transform(x)
np.mean(cross_val_score(DecisionTreeClassifier(), x_trf, y, cv=10, scoring='accuracy'))

0.6318466353677621

Not really a great example, but sometimes with the right dataset, you can see great imporvement after applying Binarization.